## Scraping scrollers

Infinite scroll sites are designed for the mobile age. Links are hard to tap with a finger on a small device,  but a simple swipe easily scrolls the page down to reveal more data. That can make scraping an infinite scroll page difficult. We’ll learn to find the actual location of the data buried in the scrolls.

Here's a couple of examples of a scrolling sites: 

- <a href="https://www.difc.ae/public-register/">DIFC Public Register</a>

- <a href="https://www.quintoandar.com.br/alugar/imovel/sao-paulo-sp-brasil">Rentals in São Paulo</a>

Let's target the data source we'll need to scrape this <a href="https://quotes.toscrape.com/scroll">mockup site</a>.







### we want ice cream!

In [1]:
pip install icecream

Note: you may need to restart the kernel to use updated packages.


In [2]:
## Lets import all the libaries we are likely to need
import requests ## to capture content from web pages
from bs4 import BeautifulSoup ## to parse our scraped data
import pandas as pd ## to easily export our data to dataframes/CSVs
from icecream import ic ## easily debug
from pprint import pprint as pp ## to prettify our printouts
import itertools ## to flatten lists
from random import randrange ## to create a range of numbers
import time # for timer
import json ## to work with JSON data

### Figure out how to scape a single page

In [3]:
url = "https://retailportal.difc.ae/api/v3/public-register/overviewList?page=1&keywords=&companyName=&registrationNumber=&type=&status=&latitude=0&longitude=0&sortBy=&difc_website=1&data_return=true&isAjax=true"

In [4]:
response = requests.get(url)
response.status_code

200

In [5]:
response.text

'{"success":true,"data":"\\n\\n<div class=\\"wrap\\">\\n    <h4><a href=\\"https:\\/\\/www.difc.ae\\/public-register\\/polyvalent-capital-holdings-limited\\">Polyvalent Capital Holdings Limited<\\/a><\\/h4>\\n    <div class=\\"row\\">\\n        <div class=\\"col-sm-3 col\\">\\n            <p class=\\"status \\">Active<\\/p>\\n            <p>Registration# 5194<\\/p>\\n            <p>Non Regulated <\\/p>\\n        <\\/div>\\n        <div class=\\"col-sm-4 col\\">\\n                            <p class=\\"location\\">\\n                    <span class=\\"icon-location\\"><\\/span>Unit OT 14-52, Level 14, Central Park Offices, Dubai International Financial Centre, Dubai, , United Arab Emirates                <\\/p>\\n                    <\\/div>\\n        <div class=\\"col-sm-3 col\\">\\n                        <!--<a class=\\"phone\\" href=\\"tel:\\"><span\\n                            class=\\"icon-phone\\"><\\/span><\\/a>-->\\n                        <!--<span class=\\"disolved\\"><\\/s

In [6]:
type(response.text)

str

In [7]:
#  method #1 slightly slower
json.loads(response.text)

{'success': True,
 'data': '\n\n<div class="wrap">\n    <h4><a href="https://www.difc.ae/public-register/polyvalent-capital-holdings-limited">Polyvalent Capital Holdings Limited</a></h4>\n    <div class="row">\n        <div class="col-sm-3 col">\n            <p class="status ">Active</p>\n            <p>Registration# 5194</p>\n            <p>Non Regulated </p>\n        </div>\n        <div class="col-sm-4 col">\n                            <p class="location">\n                    <span class="icon-location"></span>Unit OT 14-52, Level 14, Central Park Offices, Dubai International Financial Centre, Dubai, , United Arab Emirates                </p>\n                    </div>\n        <div class="col-sm-3 col">\n                        <!--<a class="phone" href="tel:"><span\n                            class="icon-phone"></span></a>-->\n                        <!--<span class="disolved"></span>-->\n                    </div>\n        <div class="col-sm-2 col">\n            <a class="pri

In [8]:
type(json.loads(response.text))

dict

In [10]:
## method 2 bit faster
content = response.json()  ## .json() is part of request library
type(content)

dict

In [12]:
content.get("data")

'\n\n<div class="wrap">\n    <h4><a href="https://www.difc.ae/public-register/polyvalent-capital-holdings-limited">Polyvalent Capital Holdings Limited</a></h4>\n    <div class="row">\n        <div class="col-sm-3 col">\n            <p class="status ">Active</p>\n            <p>Registration# 5194</p>\n            <p>Non Regulated </p>\n        </div>\n        <div class="col-sm-4 col">\n                            <p class="location">\n                    <span class="icon-location"></span>Unit OT 14-52, Level 14, Central Park Offices, Dubai International Financial Centre, Dubai, , United Arab Emirates                </p>\n                    </div>\n        <div class="col-sm-3 col">\n                        <!--<a class="phone" href="tel:"><span\n                            class="icon-phone"></span></a>-->\n                        <!--<span class="disolved"></span>-->\n                    </div>\n        <div class="col-sm-2 col">\n            <a class="primary-button blue small" hre

In [20]:
url_q = "https://quotes.toscrape.com/api/quotes?page=1"

In [21]:
response_q = requests.get(url_q)
response_q.status_code

200

In [22]:
content = response_q.json()
content

{'has_next': True,
 'page': 1,
 'quotes': [{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
   'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
  {'author': {'goodreads_link': '/author/show/1077326.J_K_Rowling',
    'name': 'J.K. Rowling',
    'slug': 'J-K-Rowling'},
   'tags': ['abilities', 'choices'],
   'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'},
  {'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles'],
   'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'},
  {'author': {'goodreads_

In [29]:
target = content.get("quotes")
type(target[9])

dict

In [31]:
target[0:2]

[{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
   'name': 'Albert Einstein',
   'slug': 'Albert-Einstein'},
  'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
  'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
 {'author': {'goodreads_link': '/author/show/1077326.J_K_Rowling',
   'name': 'J.K. Rowling',
   'slug': 'J-K-Rowling'},
  'tags': ['abilities', 'choices'],
  'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'}]

In [30]:
df = pd.DataFrame(target)
df

,author,tags,text
0,{'goodreads_link': '/author/show/9810.Albert_E...,"[change, deep-thoughts, thinking, world]",“The world as we have created it is a process ...
1,{'goodreads_link': '/author/show/1077326.J_K_R...,"[abilities, choices]","“It is our choices, Harry, that show what we t..."
2,{'goodreads_link': '/author/show/9810.Albert_E...,"[inspirational, life, live, miracle, miracles]",“There are only two ways to live your life. On...
3,{'goodreads_link': '/author/show/1265.Jane_Aus...,"[aliteracy, books, classic, humor]","“The person, be it gentleman or lady, who has ..."
4,{'goodreads_link': '/author/show/82952.Marilyn...,"[be-yourself, inspirational]","“Imperfection is beauty, madness is genius and..."
5,{'goodreads_link': '/author/show/9810.Albert_E...,"[adulthood, success, value]",“Try not to become a man of success. Rather be...
6,{'goodreads_link': '/author/show/7617.Andr_Gid...,"[life, love]",“It is better to be hated for what you are tha...
7,{'goodreads_link': '/author/show/3091287.Thoma...,"[edison, failure, inspirational, paraphrased]","“I have not failed. I've just found 10,000 way..."
8,{'goodreads_link': '/author/show/44566.Eleanor...,[misattributed-eleanor-roosevelt],“A woman is like a tea bag; you never know how...
9,{'goodreads_link': '/author/show/7103.Steve_Ma...,"[humor, obvious, simile]","“A day without sunshine is like, you know, nig..."


In [32]:
pip install cherrypicker

Note: you may need to restart the kernel to use updated packages.


In [33]:
from cherrypicker import CherryPicker

In [34]:
picker = CherryPicker(target)
type(picker)

cherrypicker.traversable.CherryPickerIterable

In [37]:
df = pd.DataFrame(best_quotes)
df

,author_goodreads_link,author_name,author_slug,tags_0,tags_1,tags_2,tags_3,text,tags_4
0,/author/show/9810.Albert_Einstein,Albert Einstein,Albert-Einstein,change,deep-thoughts,thinking,world,“The world as we have created it is a process ...,NaN
1,/author/show/1077326.J_K_Rowling,J.K. Rowling,J-K-Rowling,abilities,choices,NaN,NaN,"“It is our choices, Harry, that show what we t...",NaN
2,/author/show/9810.Albert_Einstein,Albert Einstein,Albert-Einstein,inspirational,life,live,miracle,“There are only two ways to live your life. On...,miracles
3,/author/show/1265.Jane_Austen,Jane Austen,Jane-Austen,aliteracy,books,classic,humor,"“The person, be it gentleman or lady, who has ...",NaN
4,/author/show/82952.Marilyn_Monroe,Marilyn Monroe,Marilyn-Monroe,be-yourself,inspirational,NaN,NaN,"“Imperfection is beauty, madness is genius and...",NaN
5,/author/show/9810.Albert_Einstein,Albert Einstein,Albert-Einstein,adulthood,success,value,NaN,“Try not to become a man of success. Rather be...,NaN
6,/author/show/7617.Andr_Gide,André Gide,Andre-Gide,life,love,NaN,NaN,“It is better to be hated for what you are tha...,NaN
7,/author/show/3091287.Thomas_A_Edison,Thomas A. Edison,Thomas-A-Edison,edison,failure,inspirational,paraphrased,"“I have not failed. I've just found 10,000 way...",NaN
8,/author/show/44566.Eleanor_Roosevelt,Eleanor Roosevelt,Eleanor-Roosevelt,misattributed-eleanor-roosevelt,NaN,NaN,NaN,“A woman is like a tea bag; you never know how...,NaN
9,/author/show/7103.Steve_Martin,Steve Martin,Steve-Martin,humor,obvious,simile,NaN,"“A day without sunshine is like, you know, nig...",NaN


In [35]:
best_quotes = picker.flatten().get()
best_quotes

[{'author_goodreads_link': '/author/show/9810.Albert_Einstein',
  'author_name': 'Albert Einstein',
  'author_slug': 'Albert-Einstein',
  'tags_0': 'change',
  'tags_1': 'deep-thoughts',
  'tags_2': 'thinking',
  'tags_3': 'world',
  'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
 {'author_goodreads_link': '/author/show/1077326.J_K_Rowling',
  'author_name': 'J.K. Rowling',
  'author_slug': 'J-K-Rowling',
  'tags_0': 'abilities',
  'tags_1': 'choices',
  'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'},
 {'author_goodreads_link': '/author/show/9810.Albert_Einstein',
  'author_name': 'Albert Einstein',
  'author_slug': 'Albert-Einstein',
  'tags_0': 'inspirational',
  'tags_1': 'life',
  'tags_2': 'live',
  'tags_3': 'miracle',
  'tags_4': 'miracles',
  'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as 

In [41]:
url = "https://quotes.toscrape.com/api/quotes?page=1"
r = requests.get(url)
r.status_code

200

In [42]:
test = r.json()
test

{'has_next': True,
 'page': 1,
 'quotes': [{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
   'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
  {'author': {'goodreads_link': '/author/show/1077326.J_K_Rowling',
    'name': 'J.K. Rowling',
    'slug': 'J-K-Rowling'},
   'tags': ['abilities', 'choices'],
   'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'},
  {'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
    'name': 'Albert Einstein',
    'slug': 'Albert-Einstein'},
   'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles'],
   'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'},
  {'author': {'goodreads_

In [43]:
while len(test.get("quotes")) > 0:
    print("there is content here to scrape")
    

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

...

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is content here to scrape
there is

KeyboardInterrupt: 

In [44]:
## scrape all pages

url = "https://quotes.toscrape.com/api/quotes?page={}"

In [47]:
quotes_list = []
page_number = 1
valid = 0

while valid < 1:
    link = url.format(page_number)
    ic(link)
    page_number += 1
    response = requests.get(link)
    content = response.json()
    if content.get("has_next") == False:
        valid += 1
    target = content.get("quotes")
    picker = CherryPicker(target)
    best_quotes = picker.flatten().get()
    quotes_list.append(best_quotes)
    snoozer = randrange(4, 7)
    print(f"Snoozing for {snoozer} seconds")
    time.sleep(snoozer)
print("Done scraping!")
    

ic| link: 'https://quotes.toscrape.com/api/quotes?page=1'


Snoozing for 4 seconds


ic| link: 'https://quotes.toscrape.com/api/quotes?page=2'


Snoozing for 4 seconds


ic| link: 'https://quotes.toscrape.com/api/quotes?page=3'


...

Snoozing for 5 seconds


ic| link: 'https://quotes.toscrape.com/api/quotes?page=9'


Snoozing for 6 seconds


ic| link: 'https://quotes.toscrape.com/api/quotes?page=10'


Snoozing for 4 seconds
Done scraping!


In [49]:
len(quotes_list)

10

In [50]:
quotes_flat = list(itertools.chain(*quotes_list))
len(quotes_flat)

100

In [51]:
quotes_flat[0:2]

[{'author_goodreads_link': '/author/show/9810.Albert_Einstein',
  'author_name': 'Albert Einstein',
  'author_slug': 'Albert-Einstein',
  'tags_0': 'change',
  'tags_1': 'deep-thoughts',
  'tags_2': 'thinking',
  'tags_3': 'world',
  'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
 {'author_goodreads_link': '/author/show/1077326.J_K_Rowling',
  'author_name': 'J.K. Rowling',
  'author_slug': 'J-K-Rowling',
  'tags_0': 'abilities',
  'tags_1': 'choices',
  'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'}]

In [52]:
df = pd.DataFrame(quotes_flat)
df.sample(10)

,author_goodreads_link,author_name,author_slug,tags_0,tags_1,tags_2,tags_3,text,tags_4,tags_5,tags_6,tags_7
1,/author/show/1077326.J_K_Rowling,J.K. Rowling,J-K-Rowling,abilities,choices,NaN,NaN,"“It is our choices, Harry, that show what we t...",NaN,NaN,NaN,NaN
51,/author/show/44566.Eleanor_Roosevelt,Eleanor Roosevelt,Eleanor-Roosevelt,attributed,fear,inspiration,NaN,“Do one thing every day that scares you.”,NaN,NaN,NaN,NaN
77,/author/show/82951.W_C_Fields,W.C. Fields,W-C-Fields,humor,sinister,NaN,NaN,“I am free of all prejudice. I hate everyone e...,NaN,NaN,NaN,NaN
20,/author/show/4026.Pablo_Neruda,Pablo Neruda,Pablo-Neruda,love,poetry,NaN,NaN,"“I love you without knowing how, or when, or f...",NaN,NaN,NaN,NaN
9,/author/show/7103.Steve_Martin,Steve Martin,Steve-Martin,humor,obvious,simile,NaN,"“A day without sunshine is like, you know, nig...",NaN,NaN,NaN,NaN
52,/author/show/82952.Marilyn_Monroe,Marilyn Monroe,Marilyn-Monroe,attributed-no-source,NaN,NaN,NaN,"“I am good, but not an angel. I do sin, but I ...",NaN,NaN,NaN,NaN
48,/author/show/1077326.J_K_Rowling,J.K. Rowling,J-K-Rowling,dumbledore,NaN,NaN,NaN,"“It matters not what someone is born, but what...",NaN,NaN,NaN,NaN
28,/author/show/9810.Albert_Einstein,Albert Einstein,Albert-Einstein,imagination,NaN,NaN,NaN,“Logic will get you from A to Z; imagination w...,NaN,NaN,NaN,NaN
38,/author/show/500.Jorge_Luis_Borges,Jorge Luis Borges,Jorge-Luis-Borges,books,library,NaN,NaN,“I have always imagined that Paradise will be ...,NaN,NaN,NaN,NaN
35,/author/show/209672.Charles_M_Schulz,Charles M. Schulz,Charles-M-Schulz,chocolate,food,humor,NaN,“All you need is love. But a little chocolate ...,NaN,NaN,NaN,NaN


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   author_goodreads_link  100 non-null    object
 1   author_name            100 non-null    object
 2   author_slug            100 non-null    object
 3   tags_0                 97 non-null     object
 4   tags_1                 61 non-null     object
 5   tags_2                 32 non-null     object
 6   tags_3                 19 non-null     object
 7   text                   100 non-null    object
 8   tags_4                 11 non-null     object
 9   tags_5                 7 non-null      object
 10  tags_6                 3 non-null      object
 11  tags_7                 2 non-null      object
dtypes: object(12)
memory usage: 9.5+ KB
